%md
# Prediction de cas de Covid 19 en Canada

- Felipe Gonzalez

Le but de cette projet est de predire le numero de cas des personnes contagiées par jour dans le provinces de Canada.

Le projet a 3 notebooks:

1. Collecte et integration de données
2. Traitement
3. Developpement du modele

## 3. Developpement du modèle

la variable cible est "numtoday" qui est le nombre de cas covid par jour

#### Chargement du donees

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
data_df = spark.read.csv("/FileStore/tables/Covid_DB_08Mar/ForAnalysis/CovidAnalyzed", header = True, inferSchema = True)
data_df_forPrediction = spark.read.csv("/FileStore/tables/Covid_DB_08Mar/ForAnalysis/CovidForPrediction", header = True, inferSchema = True)

In [0]:
data_df_forPrediction.select("pruid","date","date_num","numtoday").show()


+-----+----------+--------+--------+
pruid| date|date_num|numtoday|
+-----+----------+--------+--------+
 4|2021-03-07| 401.0| 300|
 1|2021-03-07| 401.0| 1299|
 2|2021-03-07| 401.0| 707|
+-----+----------+--------+--------+

### Traitement

Les valeurs nulles des colonnes météo sont mises à zéro (la plupart d'entre elles sont, par exemple, pluie mm ou neige mm)

In [0]:
df = data_df.fillna(0.0)
df_predict_tmp = data_df_forPrediction.fillna(0.0)

### Procesus d'apprendisage

#### - Selection des predicteurs

la fonction suivante crée la matrice de corrélation

In [0]:
import matplotlib.pyplot as plt
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
from pyspark.sql.functions import round

def collerationmatrix(df, columns):
  vector_col = "corr_features"
  assembler = VectorAssembler(inputCols=columns, 
                              outputCol=vector_col)
  myGraph_vector = assembler.transform(df).select(vector_col)
  matrix = Correlation.corr(myGraph_vector, vector_col).collect()[0][0]
  corrmatrix = matrix.toArray().tolist()
  df_tmp = spark.createDataFrame(corrmatrix,columns)
  for col in columns:
    df_tmp = df_tmp.withColumn(col, round(df_tmp[col], 2)) 
  df_tmp.show()

##### Correlation attributs liées au covid et numtoday (Variable cible)

- *Correlation faible avec numtoday (<0.25)*: 'numprob', 'percentrecover', 'percentoday', 'percentdeath', 'percentactive'  

- *Correlation trop fort avec numtoday (<0.25)*: 'avgtotal_last7', 'numtotal_last7', 'numtotal_last14'

- *Correlation trop fort entre eux (>0.95)*:  
  -'ratetotal_last7', 'ratetotal_last14', 'avgincidence_last7'  
  -'avgdeaths_last7', 'avgratedeaths_last7', 'numdeaths_last7', 'numdeaths_last14', 'ratedeaths_last7', 'ratedeaths_last14'  

- Les attributs totales ne sont pas prises en compte (elles peuvent également être des variables cibles):  
  - 'numtotal', 'numdeaths', 'numtested', 'numrecover'
  
- Les attributs par jour ne sont pas prises en compte (elles peuvent également être des variables cibles):  
  - 'numdeathstoday'

In [0]:
features_covidDB =  ['ratetested', 'ratetotal', 'ratedeaths', 'avgtotal_last7', 'ratetotal_last14', 'ratedeaths_last14', 'ratetotal_last7', 'ratedeaths_last7', 'rateactive', 'numtoday'] #
#[ 'avgincidence_last7', 'avgdeaths_last7', 'avgratedeaths_last7', 'percentactive', 'percentdeath', 'numtoday']

df_tmp_covidDB = df.select(features_covidDB)   
df_tmp = collerationmatrix(df_tmp_covidDB, features_covidDB)

+----------+---------+----------+--------------+----------------+-----------------+---------------+----------------+----------+--------+
ratetested|ratetotal|ratedeaths|avgtotal_last7|ratetotal_last14|ratedeaths_last14|ratetotal_last7|ratedeaths_last7|rateactive|numtoday|
+----------+---------+----------+--------------+----------------+-----------------+---------------+----------------+----------+--------+
 1.0| 0.38| 0.09| 0.62| 0.63| 0.13| 0.61| 0.14| 0.4| 0.6|
 0.38| 1.0| 0.7| 0.52| 0.66| 0.37| 0.62| 0.33| 0.48| 0.48|
 0.09| 0.7| 1.0| 0.38| 0.36| 0.35| 0.33| 0.31| 0.31| 0.36|
 0.62| 0.52| 0.38| 1.0| 0.8| 0.41| 0.81| 0.43| 0.51| 0.97|
 0.63| 0.66| 0.36| 0.8| 1.0| 0.48| 0.99| 0.51| 0.78| 0.77|
 0.13| 0.37| 0.35| 0.41| 0.48| 1.0| 0.44| 0.98| 0.7| 0.36|
 0.61| 0.62| 0.33| 0.81| 0.99| 0.44| 1.0| 0.47| 0.76| 0.79|
 0.14| 0.33| 0.31| 0.43| 0.51| 0.98| 0.47| 1.0| 0.69| 0.39|
 0.4| 0.48| 0.31| 0.51| 0.78| 0.7| 0.76| 0.69| 1.0| 0.47|
 0.6| 0.48| 0.36| 0.97| 0.77| 0.36| 0.79| 0.39| 0.47| 1.0|
+----------+---------+----------+--------------+----------------+-----------------+---------------+----------------+----------+--------+

In [0]:
features_covidDB = ['pruid', 'numrecover', 'numactive', 'rateactive', 'numtestedtoday', 'numrecoveredtoday',                      
                     'numtotal_last7',  'numdeaths_last7', 'avgtotal_last7', 'avgdeaths_last7', 'numtoday']
df_tmp_covidDB = df.select(features_covidDB)   
df_tmp = collerationmatrix(df_tmp_covidDB, features_covidDB)

+-----+----------+---------+----------+--------------+-----------------+--------------+---------------+--------------+---------------+--------+
pruid|numrecover|numactive|rateactive|numtestedtoday|numrecoveredtoday|numtotal_last7|numdeaths_last7|avgtotal_last7|avgdeaths_last7|numtoday|
+-----+----------+---------+----------+--------------+-----------------+--------------+---------------+--------------+---------------+--------+
 1.0| -0.1| -0.18| 0.2| -0.34| 0.0| -0.27| -0.3| -0.27| -0.31| -0.26|
 -0.1| 1.0| 0.11| 0.05| 0.02| 0.64| 0.24| 0.09| 0.24| 0.09| 0.22|
-0.18| 0.11| 1.0| 0.85| 0.22| -0.02| 0.71| 0.73| 0.7| 0.73| 0.67|
 0.2| 0.05| 0.85| 1.0| 0.07| -0.02| 0.51| 0.53| 0.51| 0.52| 0.47|
-0.34| 0.02| 0.22| 0.07| 1.0| 0.0| 0.42| 0.06| 0.42| 0.06| 0.42|
 0.0| 0.64| -0.02| -0.02| 0.0| 1.0| 0.0| 0.0| 0.0| 0.0| 0.0|
-0.27| 0.24| 0.71| 0.51| 0.42| 0.0| 1.0| 0.51| 1.0| 0.51| 0.97|
 -0.3| 0.09| 0.73| 0.53| 0.06| 0.0| 0.51| 1.0| 0.51| 1.0| 0.47|
-0.27| 0.24| 0.7| 0.51| 0.42| 0.0| 1.0| 0.51| 1.0| 0.51| 0.97|
-0.31| 0.09| 0.73| 0.52| 0.06| 0.0| 0.51| 1.0| 0.51| 1.0| 0.47|
-0.26| 0.22| 0.67| 0.47| 0.42| 0.0| 0.97| 0.47| 0.97| 0.47| 1.0|
+-----+----------+---------+----------+--------------+-----------------+--------------+---------------+--------------+---------------+--------+

attributs à prendre en compte: 
- 'ratetested', 'ratetotal', 'ratedeaths', 'avgtotal_last7', 'ratetotal_last7'
-  'numrecover', 'numactive', 'rateactive', 'numtestedtoday', 'numrecoveredtoday',  'avgdeaths_last7'

##### Correlation attributs liées au METEO et numtoday (Variable cible)

In [0]:
features_weather = ['pruid', "date_num", 'MaxTemp', 'MinTemp', 'MeanTemp', 'HeatDegDays', 'CoolDegDays', 'TotalRain', 'TotalPrecip', 'SnowOnGrnd', 'DirOfMaxGust', 'SpdOfMaxGust', 'numtoday']
                
df_tmp_weather = df.select(features_weather)   
df_tmp = collerationmatrix(df_tmp_weather, features_weather)

+-----+--------+-------+-------+--------+-----------+-----------+---------+-----------+----------+------------+------------+--------+
pruid|date_num|MaxTemp|MinTemp|MeanTemp|HeatDegDays|CoolDegDays|TotalRain|TotalPrecip|SnowOnGrnd|DirOfMaxGust|SpdOfMaxGust|numtoday|
+-----+--------+-------+-------+--------+-----------+-----------+---------+-----------+----------+------------+------------+--------+
 1.0| 0.01| -0.17| -0.26| -0.21| 0.19| -0.23| 0.08| -0.1| -0.04| 0.07| 0.12| -0.26|
 0.01| 1.0| -0.53| -0.5| -0.52| 0.53| -0.21| -0.11| -0.06| 0.38| 0.04| 0.01| 0.59|
-0.17| -0.53| 1.0| 0.95| 0.99| -0.98| 0.59| 0.13| 0.11| -0.56| -0.02| -0.02| -0.51|
-0.26| -0.5| 0.95| 1.0| 0.99| -0.98| 0.6| 0.18| 0.2| -0.58| -0.03| -0.02| -0.44|
-0.21| -0.52| 0.99| 0.99| 1.0| -0.99| 0.61| 0.15| 0.15| -0.58| -0.03| -0.02| -0.48|
 0.19| 0.53| -0.98| -0.98| -0.99| 1.0| -0.49| -0.16| -0.15| 0.6| 0.02| 0.01| 0.48|
-0.23| -0.21| 0.59| 0.6| 0.61| -0.49| 1.0| 0.04| 0.08| -0.21| -0.03| -0.07| -0.3|
 0.08| -0.11| 0.13| 0.18| 0.15| -0.16| 0.04| 1.0| 0.69| -0.1| 0.01| 0.15| -0.11|
 -0.1| -0.06| 0.11| 0.2| 0.15| -0.15| 0.08| 0.69| 1.0| -0.05| 0.04| 0.18| -0.05|
-0.04| 0.38| -0.56| -0.58| -0.58| 0.6| -0.21| -0.1| -0.05| 1.0| 0.08| 0.08| 0.22|
 0.07| 0.04| -0.02| -0.03| -0.03| 0.02| -0.03| 0.01| 0.04| 0.08| 1.0| 0.72| 0.07|
 0.12| 0.01| -0.02| -0.02| -0.02| 0.01| -0.07| 0.15| 0.18| 0.08| 0.72| 1.0| 0.06|
-0.26| 0.59| -0.51| -0.44| -0.48| 0.48| -0.3| -0.11| -0.05| 0.22| 0.07| 0.06| 1.0|
+-----+--------+-------+-------+--------+-----------+-----------+---------+-----------+----------+------------+------------+--------+

Il n'y a pas une bonne corrélation entre les variables météo et numtoday car il existe plusieurs valeurs nulles pour certaines provinces. On peut considérer MeanTemp dont la corrélation avec numtoday est de -0,55

##### Correlation pupulation and numtoday

In [0]:
features_pop = ['pruid', 'TousAges', 'ans80_84', 'ans85_89', 'ans90_94', 'ans95_99', 'ans100etplus', 'AgeMedian', 'numtoday']
                #'TousAges',  'ans5_9', 'ans10_14', 'ans15_19', 'ans20_24', 'ans25_29', 'ans30_34', 'ans35_39', 'ans40_44', 'ans45_49', 'ans50_54',  'ans55_59', 'ans60_64', 'ans65_69', 'ans70_74', 'ans75_79',  ]
df_tmp_population = df.select(features_pop)   
df_tmp = collerationmatrix(df_tmp_population, features_pop)

+-----+--------+--------+--------+--------+--------+------------+---------+--------+
pruid|TousAges|ans80_84|ans85_89|ans90_94|ans95_99|ans100etplus|AgeMedian|numtoday|
+-----+--------+--------+--------+--------+--------+------------+---------+--------+
 1.0| -0.95| -0.98| -0.99| -0.98| -0.99| -1.0| -0.68| -0.26|
-0.95| 1.0| 0.99| 0.99| 0.99| 0.99| 0.96| 0.43| 0.22|
-0.98| 0.99| 1.0| 1.0| 1.0| 1.0| 0.99| 0.55| 0.24|
-0.99| 0.99| 1.0| 1.0| 1.0| 1.0| 0.99| 0.55| 0.24|
-0.98| 0.99| 1.0| 1.0| 1.0| 1.0| 0.99| 0.54| 0.24|
-0.99| 0.99| 1.0| 1.0| 1.0| 1.0| 0.99| 0.55| 0.24|
 -1.0| 0.96| 0.99| 0.99| 0.99| 0.99| 1.0| 0.66| 0.25|
-0.68| 0.43| 0.55| 0.55| 0.54| 0.55| 0.66| 1.0| 0.22|
-0.26| 0.22| 0.24| 0.24| 0.24| 0.24| 0.25| 0.22| 1.0|
+-----+--------+--------+--------+--------+--------+------------+---------+--------+

Les corrélations entre les variables de population et numtoday (variable cible) sont très similaires puis qu'elles ont les mêmes valeurs pour chaque ligne d'une province.

#### - Les predicteurs:

Selecting predictors

In [0]:
#variables = [ 'pruid', 'date_num', 'numrecover','ratetested', 'ratetotal', 'ratedeaths', 
#       'numrecoveredtoday', 'numactive', 'rateactive', 'ratetotal_last7',            
#       'avgtotal_last7', 'avgdeaths_last7', 'MaxTemp',  'numtoday']


variables = [ 'pruid', 'date_num', 'numrecover','ratetotal', 
       'numrecoveredtoday', 'numactive', 'rateactive', 'ratetotal_last7',            
       'avgtotal_last7', 'avgdeaths_last7', 'MaxTemp',  'numtoday']



df_tmp = df.select(variables)
predicteurs = variables[:-1]
df_TMP = collerationmatrix(df_tmp, variables)

+-----+--------+----------+---------+-----------------+---------+----------+---------------+--------------+---------------+-------+--------+
pruid|date_num|numrecover|ratetotal|numrecoveredtoday|numactive|rateactive|ratetotal_last7|avgtotal_last7|avgdeaths_last7|MaxTemp|numtoday|
+-----+--------+----------+---------+-----------------+---------+----------+---------------+--------------+---------------+-------+--------+
 1.0| 0.01| -0.1| 0.12| 0.0| -0.18| 0.2| 0.15| -0.27| -0.31| -0.17| -0.26|
 0.01| 1.0| 0.35| 0.85| 0.0| 0.34| 0.35| 0.61| 0.61| 0.06| -0.53| 0.59|
 -0.1| 0.35| 1.0| 0.38| 0.64| 0.11| 0.05| 0.17| 0.24| 0.09| -0.23| 0.22|
 0.12| 0.85| 0.38| 1.0| 0.0| 0.41| 0.48| 0.62| 0.52| 0.22| -0.63| 0.48|
 0.0| 0.0| 0.64| 0.0| 1.0| -0.02| -0.02| 0.0| 0.0| 0.0| 0.01| 0.0|
-0.18| 0.34| 0.11| 0.41| -0.02| 1.0| 0.85| 0.63| 0.7| 0.73| -0.28| 0.67|
 0.2| 0.35| 0.05| 0.48| -0.02| 0.85| 1.0| 0.76| 0.51| 0.52| -0.31| 0.47|
 0.15| 0.61| 0.17| 0.62| 0.0| 0.63| 0.76| 1.0| 0.81| 0.37| -0.57| 0.79|
-0.27| 0.61| 0.24| 0.52| 0.0| 0.7| 0.51| 0.81| 1.0| 0.51| -0.54| 0.97|
-0.31| 0.06| 0.09| 0.22| 0.0| 0.73| 0.52| 0.37| 0.51| 1.0| -0.22| 0.47|
-0.17| -0.53| -0.23| -0.63| 0.01| -0.28| -0.31| -0.57| -0.54| -0.22| 1.0| -0.51|
-0.26| 0.59| 0.22| 0.48| 0.0| 0.67| 0.47| 0.79| 0.97| 0.47| -0.51| 1.0|
+-----+--------+----------+---------+-----------------+---------+----------+---------------+--------------+---------------+-------+--------+

In [0]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=predicteurs, outputCol="predicteurs")
df_covid = assembler.transform(df_tmp)

#### - Split Data

In [0]:
# random
train, test = df_covid.randomSplit([0.8, 0.2], 345)
# in order
#train = df_covid.filter(df_covid.date_num < (ndays*0.8))
#test = df_covid.filter(df_covid.date_num >= (ndays*0.8))

#### - Entraînement du modèle (linear)

In [0]:
from pyspark.ml.regression import LinearRegression
lr_model = LinearRegression(featuresCol="predicteurs",labelCol="numtoday")
model = lr_model.fit(train)

## 4. Evaluation du model / Test

In [0]:
evaluation_summary = model.evaluate(test)
print("MAE:{}".format(evaluation_summary.meanAbsoluteError))
print("RMSE:{}".format(evaluation_summary.rootMeanSquaredError))
print("R-squared:{}".format(evaluation_summary.r2))

MAE:124.57602714558847
RMSE:245.57969451703798
R-squared:0.915407231870093

In [0]:
n = len(predicteurs)

predictions = model.transform(test)
predictions.select(predictions.columns[n::2]).show(50)


+--------+------------------+
numtoday| prediction|
+--------+------------------+
 17| 77.43035201832858|
 66| 83.27340856272721|
 135|109.50327619011229|
 351| 181.6091775855756|
 462|334.85555447359604|
 550|420.36228182476714|
 568|488.91099630688086|
 476| 554.6741850789493|
 424|509.68236639319406|
 413| 367.7223651519995|
 460|403.85227741352725|
 383|381.26916720388465|
 197|240.11749413439648|
 178| 211.0722292178553|
 161|194.39295303829198|
 112|165.95178756915743|
 138|185.69470394468343|
 119|179.64568556894594|
 124|142.50564378879696|
 78|113.32256344123175|
 92|113.97776733282768|
 106|119.10002590830214|
 131|137.64532329718196|
 105|144.43165357202506|
 100| 142.3048747297426|
 118|147.05055551262117|
 148| 152.2648427323005|
 114|153.78178274531143|
 133| 161.8304682103341|
 251|263.29204733057827|
 700| 511.8473493857312|
 566| 656.9372480360828|
 704| 885.375844254471|
 977| 950.6841254791913|
 948| 964.8839067326569|
 1132|1051.0197150374474|
 1242| 1142.46028142668|
 1575|1338.1798259700395|
 1487| 1480.059188881712|
 1417|1459.2324296231623|
 1534|1454.2414763894012|
 1589|1466.5151846871543|
 1855|1521.5317961632545|
 1822|1554.3092998278407|
 1824|1796.6874389283014|
 1925|1852.2315244351507|
 1676| 1847.736538321972|
 1983| 1888.261997580758|
 1677| 1862.32152733504|
 1940|1867.3809183868761|
+--------+------------------+
only showing top 50 rows

## 5. Prediction

#### Prediction nombre de cas le 8 mars 2021.

In [0]:
date_min_value = df.agg({"date_num": "min"}).collect()[0][0]
date_max_value = df.agg({"date_num": "max"}).collect()[0][0]
ndays = int(date_max_value) 
print("n days: ", ndays) # 11 fevrier dans la base de données

n days: 401

In [0]:
df_forPrediction = df_predict_tmp.select(predicteurs)
day_to_predict = ndays+1

from pyspark.sql.functions import lit

df_forPrediction = df_forPrediction.withColumn("date_num", lit(day_to_predict)) # Modify by today's correspondant number

#### *Au Quebec*

In [0]:
quebec_predict = df_forPrediction.filter(df_forPrediction.pruid == 2)
assembler_predict = VectorAssembler(inputCols=predicteurs, outputCol="predicteurs")
df_predict = assembler_predict.transform(quebec_predict)
predictions_t = model.transform(df_predict)
predictions_t.select(predictions_t.columns[n::1]).show()

+--------------------+-----------------+
 predicteurs| prediction|
+--------------------+-----------------+
[2.0,402.0,275059...|626.7833621089627|
+--------------------+-----------------+

La valeur réelle au Québec est de 579 . Erreur: 8.3%

#### *En Ontario*

In [0]:
ontario_predict = df_forPrediction.filter(df_forPrediction.pruid == 1)
assembler_predict = VectorAssembler(inputCols=predicteurs, outputCol="predicteurs")
df_predict = assembler_predict.transform(ontario_predict)
predictions_t = model.transform(df_predict)
predictions_t.select(predictions_t.columns[n::1]).show()

+--------------------+------------------+
 predicteurs| prediction|
+--------------------+------------------+
[1.0,402.0,290840...|1062.6674047313284|
+--------------------+------------------+

La valeur réelle en Ontario est de 1552. Erreur: 31.5%

#### *En Alberta*

In [0]:
alberta_predict = df_forPrediction.filter(df_forPrediction.pruid == 4)
assembler_predict = VectorAssembler(inputCols=predicteurs, outputCol="predicteurs")
df_predict = assembler_predict.transform(alberta_predict)
predictions_t = model.transform(df_predict)
predictions_t.select(predictions_t.columns[n::1]).show()

+--------------------+------------------+
 predicteurs| prediction|
+--------------------+------------------+
[4.0,402.0,128974...|272.07704748903086|
+--------------------+------------------+

La valeur réelle en Alberta est de 282. Erreur: 3.5%